In [1]:
from langchain_groq import ChatGroq
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv

load_dotenv()


llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.5
)

response_schemas = [
    ResponseSchema(name="sentiment", description="Sentiment: positive, negative, or neutral", type="string"),
    ResponseSchema(name="confidence", description="Confidence score between 0.0 and 1.0", type="float")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="""You are a sentiment analysis assistant. Classify the sentiment of the input text as 'positive', 'negative', or 'neutral' and provide a confidence score between 0.0 and 1.0. If no text is provided, return: {{"sentiment": "error", "confidence": 0.0}} with the message: 'No text provided.' Format the response as JSON according to these instructions: {format_instructions}

Examples:
- Text: 'This movie is fantastic!' → {{"sentiment": "positive", "confidence": 0.9}}
- Text: 'I hated the service.' → {{"sentiment": "negative", "confidence": 0.85}}
- Text: 'The weather is okay.' → {{"sentiment": "neutral", "confidence": 0.7}}

Input text: {text}
""",
    input_variables=["text"],
    partial_variables={"format_instructions": format_instructions}
)

chain = prompt | llm | output_parser

input_text = "The new phone is incredible!"

print(f"Raw input: '{input_text}'")
print(f"Stripped input: '{input_text.strip()}'")
print(f"Is input empty? {not input_text.strip()}")

if not isinstance(input_text, str) or not input_text.strip():
    print("No text provided.")
    print({"sentiment": "error", "confidence": 0.0})
else:
    response = chain.invoke({"text": input_text})
    print("Output:", response)

Raw input: 'The new phone is incredible!'
Stripped input: 'The new phone is incredible!'
Is input empty? False
Output: {'sentiment': 'positive', 'confidence': 0.95}
